# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 9:24AM,238505,10,0085727340,ISDIN Corporation,Released
1,Jun 1 2022 9:24AM,238505,10,0085727341,ISDIN Corporation,Released
2,Jun 1 2022 9:24AM,238505,10,0085727988,ISDIN Corporation,Released
3,Jun 1 2022 9:24AM,238505,10,0085727989,ISDIN Corporation,Released
4,Jun 1 2022 9:24AM,238505,10,0085727991,ISDIN Corporation,Released
5,Jun 1 2022 9:24AM,238505,10,0085727990,ISDIN Corporation,Released
6,Jun 1 2022 9:24AM,238505,10,0085727992,ISDIN Corporation,Released
7,Jun 1 2022 9:24AM,238505,10,0085727993,ISDIN Corporation,Released
8,Jun 1 2022 9:24AM,238505,10,0085727994,ISDIN Corporation,Released
9,Jun 1 2022 9:24AM,238505,10,0085728044,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,238504,Released,7
45,238505,Released,67
46,238506,Released,1
47,238507,Released,1
48,238508,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238504,NaN,NaN,NaN,7.0
238505,NaN,NaN,NaN,67.0
238506,NaN,NaN,NaN,1.0
238507,NaN,NaN,NaN,1.0
238508,NaN,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238342,0.0,0.0,0.0,1.0
238343,0.0,0.0,0.0,1.0
238377,0.0,0.0,0.0,1.0
238380,0.0,0.0,0.0,1.0
238384,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238342,0.0,0,0,1
238343,0.0,0,0,1
238377,0.0,0,0,1
238380,0.0,0,0,1
238384,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238342,0.0,0,0,1
1,238343,0.0,0,0,1
2,238377,0.0,0,0,1
3,238380,0.0,0,0,1
4,238384,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238342,0.0,,,1
1,238343,0.0,,,1
2,238377,0.0,,,1
3,238380,0.0,,,1
4,238384,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 9:24AM,238505,10,ISDIN Corporation
67,Jun 1 2022 9:22AM,238508,12,"NBTY Global, Inc."
69,Jun 1 2022 9:20AM,238507,20,Sartorius Stedim North America
70,Jun 1 2022 9:17AM,238506,19,"Graystone, LLC"
71,Jun 1 2022 9:12AM,238504,10,ISDIN Corporation
78,Jun 1 2022 9:11AM,238503,10,ISDIN Corporation
106,Jun 1 2022 9:10AM,238499,10,ISDIN Corporation
164,Jun 1 2022 9:03AM,238501,10,ISDIN Corporation
183,Jun 1 2022 8:54AM,238500,20,Sartorius Stedim North America
184,Jun 1 2022 8:43AM,238498,102,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Jun 1 2022 9:24AM,238505,10,ISDIN Corporation,0.0,,,67
1,Jun 1 2022 9:22AM,238508,12,"NBTY Global, Inc.",0.0,,,2
2,Jun 1 2022 9:20AM,238507,20,Sartorius Stedim North America,0.0,,,1
3,Jun 1 2022 9:17AM,238506,19,"Graystone, LLC",0.0,,,1
4,Jun 1 2022 9:12AM,238504,10,ISDIN Corporation,0.0,,,7
5,Jun 1 2022 9:11AM,238503,10,ISDIN Corporation,0.0,,,28
6,Jun 1 2022 9:10AM,238499,10,ISDIN Corporation,0.0,,,58
7,Jun 1 2022 9:03AM,238501,10,ISDIN Corporation,0.0,,,19
8,Jun 1 2022 8:54AM,238500,20,Sartorius Stedim North America,0.0,,,1
9,Jun 1 2022 8:43AM,238498,102,"Senseonics, Incorporated",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 9:24AM,238505,10,ISDIN Corporation,67,,
1,Jun 1 2022 9:22AM,238508,12,"NBTY Global, Inc.",2,,
2,Jun 1 2022 9:20AM,238507,20,Sartorius Stedim North America,1,,
3,Jun 1 2022 9:17AM,238506,19,"Graystone, LLC",1,,
4,Jun 1 2022 9:12AM,238504,10,ISDIN Corporation,7,,
5,Jun 1 2022 9:11AM,238503,10,ISDIN Corporation,28,,
6,Jun 1 2022 9:10AM,238499,10,ISDIN Corporation,58,,
7,Jun 1 2022 9:03AM,238501,10,ISDIN Corporation,19,,
8,Jun 1 2022 8:54AM,238500,20,Sartorius Stedim North America,1,,
9,Jun 1 2022 8:43AM,238498,102,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 9:24AM,238505,10,ISDIN Corporation,67,,
1,Jun 1 2022 9:22AM,238508,12,"NBTY Global, Inc.",2,,
2,Jun 1 2022 9:20AM,238507,20,Sartorius Stedim North America,1,,
3,Jun 1 2022 9:17AM,238506,19,"Graystone, LLC",1,,
4,Jun 1 2022 9:12AM,238504,10,ISDIN Corporation,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 9:24AM,238505,10,ISDIN Corporation,67.0,NaN,NaN
1,Jun 1 2022 9:22AM,238508,12,"NBTY Global, Inc.",2.0,NaN,NaN
2,Jun 1 2022 9:20AM,238507,20,Sartorius Stedim North America,1.0,NaN,NaN
3,Jun 1 2022 9:17AM,238506,19,"Graystone, LLC",1.0,NaN,NaN
4,Jun 1 2022 9:12AM,238504,10,ISDIN Corporation,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 9:24AM,238505,10,ISDIN Corporation,67.0,0.0,0.0
1,Jun 1 2022 9:22AM,238508,12,"NBTY Global, Inc.",2.0,0.0,0.0
2,Jun 1 2022 9:20AM,238507,20,Sartorius Stedim North America,1.0,0.0,0.0
3,Jun 1 2022 9:17AM,238506,19,"Graystone, LLC",1.0,0.0,0.0
4,Jun 1 2022 9:12AM,238504,10,ISDIN Corporation,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4530619,336.0,14.0,21.0
102,953885,4.0,0.0,0.0
12,476969,3.0,0.0,1.0
16,238486,1.0,0.0,0.0
18,715111,3.0,0.0,0.0
19,715489,28.0,3.0,0.0
20,1192400,58.0,28.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4530619,336.0,14.0,21.0
1,102,953885,4.0,0.0,0.0
2,12,476969,3.0,0.0,1.0
3,16,238486,1.0,0.0,0.0
4,18,715111,3.0,0.0,0.0
5,19,715489,28.0,3.0,0.0
6,20,1192400,58.0,28.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,336.0,14.0,21.0
1,102,4.0,0.0,0.0
2,12,3.0,0.0,1.0
3,16,1.0,0.0,0.0
4,18,3.0,0.0,0.0
5,19,28.0,3.0,0.0
6,20,58.0,28.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,336.0
1,102,Released,4.0
2,12,Released,3.0
3,16,Released,1.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,18,19,20
Status,,,,,,,
Completed,21.0,0.0,1.0,0.0,0.0,0.0,2.0
Executing,14.0,0.0,0.0,0.0,0.0,3.0,28.0
Released,336.0,4.0,3.0,1.0,3.0,28.0,58.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,18,19,20
0,Completed,21.0,0.0,1.0,0.0,0.0,0.0,2.0
1,Executing,14.0,0.0,0.0,0.0,0.0,3.0,28.0
2,Released,336.0,4.0,3.0,1.0,3.0,28.0,58.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,18,19,20
0,Completed,21.0,0.0,1.0,0.0,0.0,0.0,2.0
1,Executing,14.0,0.0,0.0,0.0,0.0,3.0,28.0
2,Released,336.0,4.0,3.0,1.0,3.0,28.0,58.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()